In [2]:
from langchain_openai import ChatOpenAI
from src.tools import basic_eda

def run_agent(df, question):
    # Initialize the LLM
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

    # Run your EDA function
    eda = basic_eda(df)

    # Create a structured chat prompt
    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a data analysis assistant. Provide clear business insights."),
        ("human", "EDA Results: {eda}\nUser Question: {question}")
    ])

    # Format the prompt with actual values
    formatted_prompt = chat_prompt.format(eda=eda, question=question)

    # Call the model
    response = llm.invoke(formatted_prompt)

    return response.content


In [3]:
import pandas as pd

# Sample dataframe
df = pd.DataFrame({
    "sales": [100, 200, 150, 300],
    "region": ["North", "South", "East", "West"]
})

# Run the agent
insights = run_agent(df, "Which region is performing best?")
print(insights)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
from fastapi import FastAPI, UploadFile
import pandas as pd
from src.agent import run_agent

app = FastAPI(title='Agentic AI Data Analysis Assistant')

@app.post("/analyze")
def analyze(data: dict):
    df = data["df"]
    question = data["question"]
    insight = run_agent(df, question)
    return JSONResponse(content={"insight": insight})
from langchain_openai import ChatOpenAI
from src.tools import basic_eda

def run_agent(df, question):
    llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
    eda = basic_eda(df)
    prompt = f'''
You are a data analysis assistant.
EDA Results: {eda}
User Question: {question}
Generate clear business insights.
'''
    return llm.predict(prompt)
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
import pandas as pd

def basic_eda(df):
    return {
        "shape": df.shape,
        "columns": list(df.columns),
        "missing": df.isnull().sum().to_dict(),
        "describe": df.describe(include='all').to_dict()
    }
import streamlit as st, requests

API='http://127.0.0.1:8000/analyze'

st.title('📊 Agentic AI Data Analysis Assistant')

file = st.file_uploader('Upload CSV', type='csv')
question = st.text_input('What insight do you want?')

if st.button('Analyze') and file:
    files = {'file': file}
    params = {'question': question}
    r = requests.post(API, files=files, params=params)
    st.write(r.json()['insight'])
